# Packages installation

In [ ]:
#install packages needed 
!pip install matplotlib
!pip install particles

# Importations

In [1]:
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sb
#import scipy.stats as stats

# Modules from particles
import particles 
from particles import distributions as dists

# A word about what we are trying to achieve

Generally, for the type of problems we are speaking about, we deal with:
- An hidden variable $X_t$ that follows a markov process. This variable can be multivariate. It is characterized by a **transition kernel**.
- An observed variable $Y_t$ whose distribution depends on $X_t$. It is characterized by an **emission law**. 

The basic task is, given that we *know* the transition kernel, the emission law, and all their parameters, to recover the $X_t$ based on a sequence of $Y_t$ (*filtering* / *complete smoothing*).

A more challening tasks is to perform **bayesian inference**, that is, to estimate the posterior of the parameters given the data (and prior on the parameters). This task relies on particle MCMC algorithms. 

In our setting: 

- $X$ is composed of 
    - $u_t$, the expression level. It is the variable that, in real life, researchers are trying to recover. 
    - $s_t$, the local scaling term. A variable that follows a markov process independently from everything else and will be useful at some point. 
    - We might even add $a_t$ and $x_t$
- $Y$ is simply $y_t$, the read counts. It is basically **a noisy observation of $u_t$**. Its distribution is described by the **emission law** that involves $u_t$ and $s_t$. 

## What we have to do 

Given the level of complexity of the model, our assignment has changed. We can focus on:

1. Creating the model
    - Having a working subclass of Feynman Kac 
    - being able to generate data (should not be too hard)

2. Using the bootstrap filter

3. Using the PMMH algorithm to perform some bayesian inference

## How we are going to do it 

Since our model is pretty complicated, the previous approach subclassing (ssm.StateSpaceModel) will not work. 
Instead, we are going to define ourselves a Feynman-Kac model as explained here: 
https://particles-sequential-monte-carlo-in-python.readthedocs.io/en/latest/notebooks/Defining_Feynman-Kac_models_manually.html

- In the method M, we are going to define a way to sample from our X
    - And we use a function to do so, in order to use it as well when generating the data
- In the method logG, we are going to compute the loglikelihood 

!Note that xp and x are tables containing N particle (we will have to loop throught them)

In [7]:
# The simple example of a feynman-kac model from the package

class GaussianProb(particles.FeynmanKac):
    def __init__(self, a=0., b=1., T=10):
        self.a, self.b, self.T = a, b, T

    def M0(self, N):
        return stats.norm.rvs(size=N)

    def M(self, t, xp):
        return stats.norm.rvs(loc=xp, size=xp.shape)

    def logG(self, t, xp, x):
        return np.where((x < self.b) & (x > self.a), 0., -np.inf)

## Various notes and tips

- We can further simplify the model if it is too complicated
- We could use **numba** (a package) to make the loops run faster
- Compute the loglikelihood, not the likelihood. When we have to sum likelihood, it is always possible to use the log_sum_exp trick
- Simulate the data using parameter's value that make sens (look in the articles) or go looking for a datasource (the Parseq module might have some)
- For the initial law, find it in the paper or use something that makes sens


## Choice regarding the drifts
The explanations in the supplementary materials are unclear. What we are chosing to do is:

**Upward drift**

$u_{t+1} = u_t + Z, \quad Z \sim \mathcal{E}(\frac{\lambda_u}{u_t})$

**Downward drift**

$u_{t+1} = u_t - Z, \quad Z \sim \mathcal{E}(\frac{\lambda_d}{u_t})$

**In both cases, the exponential is parametrized by its RATE** (whereas the numpy funciton works in terms of scale)

**Be wary of the parametrization of the gamma distribution !**
- The gamma class from particles.dists uses a RATE parameter b (a SCALE parameter 1/b)
- In the supplementary materials, they are using a SCALE theta !

## Regarding the emission law

**We have two definitions of it**
- The 1st will be useful to generate the data 
- The 2nd will be useful to compute the likelihood (we can truncate the sum at 10 or 30)

**Other tips**
- Likelihod of a truncated negative binomial: compute it the usual way, then divide by (1 - probability of 0)
- Poisson law truncated in 0: keep generating until you gave a non-zero value

<h1 align = "center"> TO DO </h1>

### Make a choice regarding the initial law 
Both for generating the data and for M0 in the Feynman-Kac model.
It could make sense to use [0,0] as an itial law. Indeed:
- For u_t, it simply means we start in a non-expressed region
- For s_t, given it follows a gamma with probability 1/2, starting at 0 is not really an issue

### Find values of the parameter that "make sens"
See table S2 in the supplementary materials (p17). This is to ensure that the data generated are not totally stupid

In [ ]:
def stepX(xp, eta, alpha, zeta, beta, beta_0, gamma_u, gamma_d, lambda_u, lambda_d,
            alpha_s, kappa_s):
    """
    Performs a step of the kernel
    """
    
    rng = np.random.default_rng(12345) #A numpy generator 
    
    x = np.zeros(shape = (2))
    # Computations for u_t
    uniu = dists.Uniform(a = 0., b = 1.).rvs(size = 1) # Uniform to "choose a move"
        
    if xp[0] == 0 :
        if uniu <= eta:
            # Dirac on xp = 0 
            x[0] = 0
        else:
            # Exponential law with rate zeta 
            x[0] = rng.exponential(scale = 1 / zeta, size = 1)
            
    elif xp[0] !=0:
        if uniu <= alpha:
            # Dirac on xp
            x[0] = xp[0]

        elif uniu <= alpha + beta:
            # Exponential law with rate zeta 
            x[0] = rng.exponential(scale = 1 / zeta, size = 1)

        elif uniu <= alpha + beta + beta_0:
            # Dirac on 0
            x[0] = 0

        elif uniu <= alpha + beta + beta_0 + gamma_u:
            # Drift upward (rate lambda_u/xp[0])
            x[0] = xp[0] + 

        else:
            # Drift downward (rate lambda_d/xp[0])
            x[0] = xp[0] - rng.exponential(scale = xp[0]/lambda_d, size = 1)
        
        
        # Computations for s_t
        unis = dists.Uniform(a = 0., b = 1.).rvs(size = 1) # Uniform to "choose" a move 
               
        if unis <= alpha_s:
            x[1] = xp[1]
        else:
            x[1] = dists.Gamma(a = kappa_s, b = kappa_s).rvs(size = 1))
            
        return x 

def sampleY(x, kappa, theta, epsilon_b, epsilon_0, b):
    """
    Samples an observation y_t based on the vector x_t and some parameters
    Uses the 1st expression of the emission model 
    """
    
    # Sample the number of molecules initially sampled (x_t)
    x_t = dists.Poison(rate = x[0]*x[1]/(kappa*theta).rvs(size = 1)
                           
    # Sample the amplificaiton coefficient (a_t)
    a_t = dists.Gamma(kappa, 1/theta)
                       
    # Sample the read counts (y_t)
    uniy = dists.Uniform(a = 0., b = 1.).rvs(size = 1) # Uniform to "choose a move"
    
    # With proba "1 - epsilon_b - epsilon_0"
    if uniy < 1 - epsilon_b - epsilon_0:
        y = dists.Poisson(rate = x_t*a_t).rvs(size = 1)
    
    # With proba "epsilon_b"
    elif uniy < 1 - epsilon_0:
        # Generate following a Poisson distribution truncated in 0               
        proposition = 0 
        while proposition == 0:
            proposition = dists.Poisson(rate = 1*a_t).rvs(size = 1)
        y = proposition
                       
    # With proba "epsilon_0"
    else:
        y = dists.DiscreteUniform(0, b+1) # In the supplementary materials, they stae that b should be equal to max(y)
                                          # so we have to do a choice here
    return y 

**I don't know what to choose as initial law, for now it is just a Dirac on (0,0)**

In [ ]:
def generate_data(length, 
                  eta, alpha, zeta, beta, beta_0, gamma_u, gamma_d, lambda_u, lambda_d,
                alpha_s, kappa_s,
                 kappa, theta, b):
    """
    Function that allows to generate data (that we can then analyze)
    length  is the length of the data to be generated
    """
    
    # Initialisation 
    X = np.zeros(shape = (length, 2))
    Y = np.zeros(shape = length)
    
    # Time = 0
    X[0] = np.array([0,0]) 
    Y[0] = sampleY(X[0], kappa =kappa, theta = theta, epsilon_b = epsilon_b, epsilon_0 = epsilon_0, b = b)
    
    # Time = 1, ...... 
    for time in range(1, length):
        # Update X
        X[time] = stepx(X[time - 1], eta = eta, alpha = alpha, zeta = zeta, beta = beta, beta_0 = beta_0, 
                        gamma_u = gamma_u, gamma_d=gamma_d, lambda_u = lambda_u, lambda_d = lambda_d,
                        alpha_s = alpha_s, kappa_s = kappa_s)
        # Sample from Y|X
        Y[time] = ssampleY(X[time], kappa =kappa, theta = theta, epsilon_b = epsilon_b, epsilon_0 = epsilon_0, b = b)
    
    return X, Y

In [6]:
class RnaProb(particles.FeynmanKac):
    
    
    def __init__(self, a = 0.): # ALl parameters should be added there
        self.a = a # And so on for all parameters 
                 
    def M0(self, N):
    
    def M(self, t, xp):
                 
        # Initialize empty vector
        # It has two columns, one for u_t and one for s_t
        x = np.zeros(shape = (N, 2)) # Maybe this N won't work
        
        # Loop over the particles
        for i in range(N):
            # Call the function that does one move using the kernel of x
            x[i,:] = stepX(xp[i,:], 
                          eta, alpha, zeta, beta, beta_0, gamma_u, gamma_d, lambda_u, lambda_d,
                          alpha_s, kappa_s)
            
        return x
    
                 
    def logG(self, t, xp, x):
                 
                 
        

SyntaxError: invalid syntax (2551365168.py, line 1)

In [17]:
x = np.zeros(shape = (2))
x.shape

(2,)

# Archives

In [ ]:
## Should probably be transformed into a function so that it can be reused for the sampling
                 
            # Computations for u_t
            uniu = dists.Uniform(a = 0., b = 1.).rvs(size = 1) # Uniform to "choose a move"
        
            if xp[i, 0] == 0 :
                if uniu <= self.eta:
                # Dirac on xp = 0 
                    x[i,0] = 0
                else:
                    # Exponential law with rate zeta 
                    x[i,0] = dists.Gamma(1, self.zeta).rvs(size = 1)
            
            elif xp[i,0] !=0:
                if uniu <= self.alpha:
                # Dirac on xp
                    x[i,0] = xp[i, 0]

                elif uniu <= self.alpha + self.beta:
                    # Exponential law with rate zeta 
                    x[i,0] = dists.Gamma(1, self.zeta).rvs(size = 1)

                elif uniu <= self.alpha + self.beta + self.beta_0:
                    # Dirac on 0
                    x[i,0] = 0

                elif uniu <= self.alpha + self.beta + self.beta_0 + self.gamma_u:
                    # Drift upward
                    x[i,0] = xp[i,0] + dists.Gamma(1, self.lambda_u/xp[i,0]).rvs(size = 1)

                else:
                    # Drift downward 
                    x[i,0] = xp[i,0] - dists.Gamma(1, self.lambda_d/xp[i,0]).rvs(size = 1)
            
                # Computations for s_t
                unis = dists.Uniform(a = 0., b = 1.).rvs(size = 1) # Uniform to "choose" a move 
               
                if unis <= self.alpha_s:
                    x[i,1] = xp[i,1]
                else:
                    x[i,1] = dists.Gamma(a = self.kappa_s, b = self.kappa_s).rvs(size = 1)
                 